In [173]:
import pandas_datareader.data as web
from functools import reduce
import datetime
import pandas as pd


In [174]:
stocks = {'AAPL': 0.41709, 'CAH': 0.0, 'CMCSA': 0.00698, 'DISH': 0.01731, 'GOOG': 0.00761, 'HSY': 0.19678, 'JNJ': 0.0, 'JPM': 0.0, 'K': 0.0, 'MA': 0.17018, 'NFLX': 0.17273, 'UL': 0.0, 'WBA': 0.01132}

# Start and ending times for dataset
startTrain = datetime.datetime(2009, 1, 1)
endTrain = datetime.datetime(2017, 1, 1)

# testing dataset
startTest = datetime.datetime(2017, 1, 2)
endTest = datetime.datetime(2019, 1, 1)

### Steps to get the cumulative percentage change of portfolio assuming that the portfolio is rebalanced to the above weightage every 10 days
1. Get the 10 day percentage return for each stock
2. calculate the percentage change for whole portfolio according to the above weightage for each 10 day period
3. Get the cumulative percentage change for whole portfolio throughout the entire period
4. This assumes that the portfolio is rebalanced to the weightage outlined in the weights dictionary above

In [175]:
def get_stock(ticker, start=startTrain, end=endTrain):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    # print(data)
    data[f'{ticker}'] = data["Close"]#(data["Close"] - data["Open"])/data["Open"]
    data = data[[f'{ticker}']] 
    # print(data.head())
    return data 

In [176]:
def combine_stocks(tickers, testing=False):
    data_frames = []
    for i in tickers:
        if (testing):
            data_frames.append(get_stock(i, startTest, endTest))
        else:
            data_frames.append(get_stock(i))
        
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='outer'), data_frames)

    # print(df_merged.head())
    return df_merged

In [177]:
portfolio = combine_stocks(stocks.keys())

In [178]:
portfolio

,AAPL,CAH,CMCSA,DISH,GOOG,HSY,JNJ,JPM,K,MA,NFLX,UL,WBA
Date,,,,,,,,,,,,,
2008-12-31,3.048214,34.470001,8.440000,11.090000,153.250580,34.740002,59.830002,31.530001,43.849998,14.293000,4.270000,23.020000,24.670000
2009-01-02,3.241071,35.220001,8.995000,11.740000,160.060059,35.810001,60.650002,31.350000,45.049999,14.978000,4.267143,23.709999,25.549999
2009-01-05,3.377857,35.730000,8.515000,11.490000,163.412491,35.490002,60.049999,29.250000,44.790001,15.326000,4.562857,23.490000,26.840000
2009-01-06,3.322143,35.630001,8.615000,11.980000,166.406265,35.970001,59.689999,29.879999,44.650002,16.202000,4.705714,23.889999,26.709999
2009-01-07,3.250357,35.939999,8.320000,12.020000,160.403763,35.419998,59.130001,28.090000,44.470001,15.207000,4.672857,23.450001,27.030001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-23,29.129999,72.879997,35.325001,58.889999,789.909973,103.620003,115.959999,87.050003,73.570000,104.709999,125.589996,40.470001,84.279999
2016-12-27,29.315001,73.070000,35.435001,59.549999,791.549988,103.800003,115.910004,87.129997,73.849998,104.800003,128.350006,40.520000,84.489998
2016-12-28,29.190001,72.550003,35.224998,58.950001,785.049988,103.150002,115.099998,86.500000,73.430000,103.680000,125.889999,40.270000,83.589996


In [179]:
indexes = [i for i in range(1,2016)]
portfolio['indexNum'] = indexes
rowsToBeRemoved = []

for index, row in portfolio.iterrows():
    if row.indexNum % 10 != 0:
        rowsToBeRemoved.append(index)

portfolio = portfolio.drop(rowsToBeRemoved)
portfolio = portfolio.drop(columns=['indexNum'])
portfolio

,AAPL,CAH,CMCSA,DISH,GOOG,HSY,JNJ,JPM,K,MA,NFLX,UL,WBA
Date,,,,,,,,,,,,,
2009-01-14,3.047500,37.360001,7.405000,12.330000,149.923050,34.110001,57.950001,25.910000,42.650002,14.530000,4.220000,22.570000,26.139999
2009-01-29,3.321429,37.830002,7.665000,13.070000,171.018982,37.840000,58.279999,25.430000,45.119999,12.909000,5.268571,22.530001,27.889999
2009-02-12,3.545357,38.389999,6.890000,13.200000,180.847137,36.540001,57.779999,26.190001,41.310001,16.180000,5.418571,20.440001,26.990000
2009-02-27,3.189643,32.450001,6.530000,11.250000,168.363937,33.689999,50.000000,22.850000,38.919998,15.803000,5.177143,19.280001,23.860001
2009-03-13,3.426071,31.540001,6.515000,10.880000,161.604263,33.209999,50.639999,23.750000,36.990002,15.831000,5.624286,18.219999,23.879999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-10-26,28.897499,74.400002,31.280001,58.230000,799.070007,96.150002,114.559998,69.129997,74.070000,103.220001,126.970001,41.770000,82.300003
2016-11-09,27.719999,69.250000,31.725000,57.930000,785.309998,100.459999,120.309998,73.250000,74.879997,106.190002,122.190002,41.299999,81.540001
2016-11-23,27.807501,70.550003,34.480000,55.689999,760.989990,97.160004,113.070000,78.860001,73.389999,104.559998,117.690002,39.099998,84.050003


In [180]:
indexes = [i for i in range(201)]
portfolio['indexNum'] = indexes
portfolio = portfolio.set_index('indexNum')
portfolioDailyChange = portfolio

aaplDailyChange = [0]
cahDailyChange = [0]
cmcsaDailyChange = [0]
dishDailyChange = [0]
googDailyChange = [0]
hsyDailyChange = [0]
jnjDailyChange = [0]
jpmDailyChange = [0]
kDailyChange = [0]
maDailyChange = [0]
nflxDailyChange = [0]
ulDailyChange = [0]
wbaDailyChange = [0]

for index, row in portfolioDailyChange.iterrows():
    if index==0:
        continue
    previousVal = portfolioDailyChange.iloc[[index-1]]['AAPL']
    percentageChange = (row['AAPL'] - previousVal)/previousVal*100
    aaplDailyChange.append(float(percentageChange))
    
    previousVal = portfolioDailyChange.iloc[[index-1]]['CAH']
    percentageChange = (row['CAH'] - previousVal)/previousVal*100
    cahDailyChange.append(float(percentageChange))

    previousVal = portfolioDailyChange.iloc[[index-1]]['CMCSA']
    percentageChange = (row['CMCSA'] - previousVal)/previousVal*100
    cmcsaDailyChange.append(float(percentageChange))


    previousVal = portfolioDailyChange.iloc[[index-1]]['DISH']
    percentageChange = (row['DISH'] - previousVal)/previousVal*100
    dishDailyChange.append(float(percentageChange))


    previousVal = portfolioDailyChange.iloc[[index-1]]['GOOG']
    percentageChange = (row['GOOG'] - previousVal)/previousVal*100
    googDailyChange.append(float(percentageChange))

    previousVal = portfolioDailyChange.iloc[[index-1]]['HSY']
    percentageChange = (row['HSY'] - previousVal)/previousVal*100
    hsyDailyChange.append(float(percentageChange))


    previousVal = portfolioDailyChange.iloc[[index-1]]['JNJ']
    percentageChange = (row['JNJ'] - previousVal)/previousVal*100
    jnjDailyChange.append(float(percentageChange))


    previousVal = portfolioDailyChange.iloc[[index-1]]['JPM']
    percentageChange = (row['JPM'] - previousVal)/previousVal*100
    jpmDailyChange.append(float(percentageChange))


    previousVal = portfolioDailyChange.iloc[[index-1]]['K']
    percentageChange = (row['K'] - previousVal)/previousVal*100
    kDailyChange.append(float(percentageChange))


    previousVal = portfolioDailyChange.iloc[[index-1]]['MA']
    percentageChange = (row['MA'] - previousVal)/previousVal*100
    maDailyChange.append(float(percentageChange))


    previousVal = portfolioDailyChange.iloc[[index-1]]['NFLX']
    percentageChange = (row['NFLX'] - previousVal)/previousVal*100
    nflxDailyChange.append(float(percentageChange))


    previousVal = portfolioDailyChange.iloc[[index-1]]['UL']
    percentageChange = (row['UL'] - previousVal)/previousVal*100
    ulDailyChange.append(float(percentageChange))

    
    previousVal = portfolioDailyChange.iloc[[index-1]]['WBA']
    percentageChange = (row['WBA'] - previousVal)/previousVal*100
    wbaDailyChange.append(float(percentageChange))

    

# print(float(aaplDailyChange[1]))
portfolioDailyChange['AAPLPercentageChange'] = aaplDailyChange
portfolioDailyChange['CAHPercentageChange'] = cahDailyChange
portfolioDailyChange['CMCSAPercentageChange'] = cmcsaDailyChange
portfolioDailyChange['DISHPercentageChange'] = dishDailyChange
portfolioDailyChange['GOOGPercentageChange'] = googDailyChange
portfolioDailyChange['HSYPercentageChange'] = hsyDailyChange
portfolioDailyChange['JNJPercentageChange'] = jnjDailyChange
portfolioDailyChange['JPMPercentageChange'] = jpmDailyChange
portfolioDailyChange['KPercentageChange'] = kDailyChange
portfolioDailyChange['MAPercentageChange'] = maDailyChange
portfolioDailyChange['NFLXPercentageChange'] = nflxDailyChange
portfolioDailyChange['ULPercentageChange'] = ulDailyChange
portfolioDailyChange['WBAPercentageChange'] = wbaDailyChange


portfolioDailyChange = portfolioDailyChange.drop(columns=['AAPL', 'CAH', 'CMCSA', 'DISH', 'GOOG', 'HSY', 'JNJ', 'JPM', 'K', 'MA', 'NFLX', 'UL', 'WBA'])
portfolioDailyChange

,AAPLPercentageChange,CAHPercentageChange,CMCSAPercentageChange,DISHPercentageChange,GOOGPercentageChange,HSYPercentageChange,JNJPercentageChange,JPMPercentageChange,KPercentageChange,MAPercentageChange,NFLXPercentageChange,ULPercentageChange,WBAPercentageChange
indexNum,,,,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,8.988651,1.258033,3.511138,6.001620,14.071173,10.935208,0.569453,-1.852565,5.791318,-11.156224,24.847658,-0.177222,6.694721
2,6.741917,1.480300,-10.110895,0.994645,5.746821,-3.435516,-0.857927,2.988597,-8.444144,25.338909,2.847074,-9.276521,-3.226962
3,-10.033237,-15.472776,-5.224959,-14.772726,-6.902625,-7.799678,-13.464865,-12.752959,-5.785532,-2.330036,-4.455564,-5.675146,-11.596885
4,7.412367,-2.804314,-0.229714,-3.288888,-4.014918,-1.424754,1.279999,3.938729,-4.958881,0.177181,8.636869,-5.497932,0.083816
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,-1.491393,-1.834014,-4.107904,4.730218,1.644744,1.263828,-2.890568,1.467782,-3.125817,1.835044,27.608041,-7.527115,5.108557
197,-4.074746,-6.922045,1.422633,-0.515197,-1.722003,4.482577,5.019204,5.959790,1.093557,2.877350,-3.764668,-1.125212,-0.923453
198,0.315662,1.877261,8.684000,-3.866739,-3.096867,-3.284885,-6.017786,7.658704,-1.989848,-1.534989,-3.682789,-5.326878,3.078246


In [181]:
portfolioChangesArr = []

for index, row in portfolioDailyChange.iterrows():
  if index==0:
    continue
  portfolioChange = row['AAPLPercentageChange'] * stocks['AAPL'] + row['CAHPercentageChange'] * stocks['CAH'] +  row['CMCSAPercentageChange'] * stocks['CMCSA'] +  row['DISHPercentageChange'] * stocks['DISH'] +  row['GOOGPercentageChange'] * stocks['GOOG'] +  row['HSYPercentageChange'] * stocks['HSY'] +  row['JNJPercentageChange'] * stocks['JNJ'] +  row['JPMPercentageChange'] * stocks['JPM'] +  row['KPercentageChange'] * stocks['K'] +  row['MAPercentageChange'] * stocks['MA'] +  row['NFLXPercentageChange'] * stocks['NFLX'] +  row['ULPercentageChange'] * stocks['UL'] +  row['WBAPercentageChange'] * stocks['WBA']
  
  portfolioChangesArr.append(portfolioChange)
    # print(row['AAPLPercentageChange'])
    
portfolioChangesArr

[8.605537921419351,
 6.893743169726329,
 -7.3617104828384585,
 4.245121148626625,
 7.26381464544009,
 11.440577436261254,
 -0.6537216417267501,
 1.1852371347850321,
 -0.743464900243609,
 3.278393214205698,
 -3.683606161930908,
 1.8819238987646745,
 12.422516879306357,
 4.119650533205785,
 -0.3110631667392809,
 -1.1073362063929257,
 9.059204749486623,
 -2.4346765519572884,
 6.6642032854995765,
 2.409636604270606,
 2.7761782469804293,
 -1.6357542725190548,
 -0.8486785422239713,
 5.287515761713164,
 -2.2347293345645576,
 -0.44765006462867407,
 -1.1874595016623573,
 3.849053421600778,
 9.735102017343225,
 0.9403683470870515,
 6.581806631738203,
 11.24615281803475,
 -10.838699777845825,
 1.9234252317895892,
 2.7867165990322658,
 7.153327276538041,
 -7.2534433139302354,
 3.703762799698072,
 -2.383479433218463,
 4.243727212696669,
 -3.9898877685853176,
 8.01843097944781,
 8.190991689891286,
 2.1412997086302803,
 4.287625194450612,
 1.4862574852725294,
 -0.8080242178842277,
 1.8556457857620499

In [182]:
cumulativeChange = 1
for i in portfolioChangesArr:
    cumulativeChange *= 1 + (i/100)
    
cumulativeChange

11.277510009834252

In [183]:
totalChangeArr = []
for i, value in enumerate(portfolioChangesArr):
    totalChangeArr.append(1 + (value/100))

for i, value in enumerate(totalChangeArr):
    if i == 0:
        continue
    totalChangeArr[i] = totalChangeArr[i-1] * value
totalChangeArr

[1.0860553792141936,
 1.1609252477382175,
 1.0754612920775548,
 1.1211159268328323,
 1.202551709718477,
 1.340130569279903,
 1.3313698457211245,
 1.3471497355339412,
 1.337134150096522,
 1.3809706653381133,
 1.3301011448152602,
 1.355132636137281,
 1.523474216598424,
 1.5862360302857736,
 1.5813018342580074,
 1.563791506514913,
 1.7054585809451805,
 1.6639361807715647,
 1.7748242703991595,
 1.8175910856801762,
 1.8680506540198845,
 1.8374939356339342,
 1.8218995188875422,
 1.918232743111297,
 1.8753654332957665,
 1.8669703587215942,
 1.844800841803735,
 1.9158082117269015,
 2.1023140955951543,
 2.1220835919064807,
 2.2617550304896095,
 2.5161154575880595,
 2.243401257076118,
 2.286551402905005,
 2.350271110395164,
 2.5183936948076546,
 2.3357234357331884,
 2.4222330914497037,
 2.3644996638903883,
 2.4648425795710263,
 2.366498126973839,
 2.5562541459151613,
 2.7656367105795736,
 2.824857281404986,
 2.9459765739097787,
 2.989761371253888,
 2.965603375317209,
 3.0206344693737,
 2.9682241

In [184]:
def std_dev(data):
    # Get number of observations
    n = len(data)
    # Calculate mean
    mean = sum(data) / n
    # Calculate deviations from the mean
    deviations = sum([(x - mean)**2 for x in data])
    # Calculate Variance & Standard Deviation
    variance = deviations / (n - 1)
    s = variance**(1/2)
    return s

# Sharpe Ratio From Scratch
def sharpe_ratio(data, risk_free_rate=0.02): # risk_free_rate refers to 10 year treasury bond rate(2% is used)
    # Calculate Average Daily Return
    mean_daily_return = sum(data) / len(data)
    # Calculate Standard Deviation
    s = std_dev(data)
    # Calculate Daily Sharpe Ratio
    daily_sharpe_ratio = (mean_daily_return - risk_free_rate/252) / s #TODO: check if risk_free_rate should be divided by 252 trading days
    # Annualize Daily Sharpe Ratio
    sharpe_ratio = 252**(1/2) * daily_sharpe_ratio
    
    return sharpe_ratio

In [185]:
for i, value in enumerate(portfolioChangesArr):
    portfolioChangesArr[i] = value/100

sharpe_ratio(portfolioChangesArr)

5.088729683858909